<a href="https://colab.research.google.com/github/sergiu123456789/Agentic-AI/blob/main/Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install prophet
!pip install yfinance pandas --quiet

Graph the results

In [45]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Suppress Prophet logging
import logging
logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)


def download_stock_data(symbol="GOOG", period="5y"):
    try:
        df = yf.download(symbol, period=period, interval="1d", auto_adjust=True)
        if df.empty:
            return None, None
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join(col).strip() for col in df.columns.values]
            close_col = f'Close_{symbol}'
        else:
            close_col = 'Close'
        return df, close_col
    except Exception as e:
        return None, None


def calculate_forecast_metrics(actual, predicted):
    mae = abs(actual - predicted).mean()
    mape = (abs((actual - predicted) / actual) * 100).mean()
    return mae, mape


def create_prophet_forecasts(df, close_col, periods=365):
    prophet_df = df[[close_col]].reset_index()
    prophet_df.rename(columns={'Date': 'ds', close_col: 'y'}, inplace=True)

    train_size = len(prophet_df) - 30
    train_df = prophet_df[:train_size].copy()
    test_df = prophet_df[train_size:].copy()

    # Forecast 1
    m1 = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
    m1.fit(train_df)
    forecast_test1 = m1.predict(m1.make_future_dataframe(periods=30))
    test_predictions1 = forecast_test1.tail(30)['yhat'].values
    test_actual = test_df['y'].values
    mae1, mape1 = calculate_forecast_metrics(test_actual, test_predictions1)
    m1_full = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
    m1_full.fit(prophet_df)
    forecast1 = m1_full.predict(m1_full.make_future_dataframe(periods=periods))

    # Forecast 2
    m2 = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True, changepoint_prior_scale=0.5)
    m2.fit(train_df)
    forecast_test2 = m2.predict(m2.make_future_dataframe(periods=30))
    test_predictions2 = forecast_test2.tail(30)['yhat'].values
    mae2, mape2 = calculate_forecast_metrics(test_actual, test_predictions2)
    m2_full = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True, changepoint_prior_scale=0.5)
    m2_full.fit(prophet_df)
    forecast2 = m2_full.predict(m2_full.make_future_dataframe(periods=periods))

    # Get forecast values
    current_price = float(prophet_df['y'].iloc[-1])
    future_forecast1 = forecast1.iloc[-periods:]
    future_forecast2 = forecast2.iloc[-periods:]

    def get_forecast_values(forecast, days):
        if len(forecast) > days-1:
            return float(forecast.iloc[days-1]['yhat'])
        else:
            return float(forecast.iloc[-1]['yhat'])

    f1_30d = get_forecast_values(future_forecast1, 30)
    f1_90d = get_forecast_values(future_forecast1, 90)
    f1_365d = float(future_forecast1.iloc[-1]['yhat'])

    f2_30d = get_forecast_values(future_forecast2, 30)
    f2_90d = get_forecast_values(future_forecast2, 90)
    f2_365d = float(future_forecast2.iloc[-1]['yhat'])

    def pct_change(val):
        return float(((val - current_price) / current_price) * 100)

    forecast1.attrs = {
        'mae': float(mae1), 'mape': float(mape1), 'current_price': current_price,
        'forecast_30d': f1_30d, 'pct_30d': pct_change(f1_30d),
        'forecast_90d': f1_90d, 'pct_90d': pct_change(f1_90d),
        'forecast_365d': f1_365d, 'pct_365d': pct_change(f1_365d)
    }

    forecast2.attrs = {
        'mae': float(mae2), 'mape': float(mape2), 'current_price': current_price,
        'forecast_30d': f2_30d, 'pct_30d': pct_change(f2_30d),
        'forecast_90d': f2_90d, 'pct_90d': pct_change(f2_90d),
        'forecast_365d': f2_365d, 'pct_365d': pct_change(f2_365d)
    }

    return forecast1, forecast2


def create_interactive_chart(df, close_col, forecast1, forecast2, symbol="GOOG"):
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[0.75, 0.25],
        vertical_spacing=0.08,
        subplot_titles=(f"{symbol} Close Price & Forecasts", "Forecast Summary"),
        specs=[[{"type": "xy"}], [{"type": "table"}]]
    )

    current_price = forecast1.attrs['current_price']

    # Historical price
    fig.add_trace(go.Scatter(
        x=df.index, y=df[close_col], mode='lines',
        name='Historical Close',
        line=dict(color='#1f77b4', width=3)
    ), row=1, col=1)

    # Forecast lines
    forecast1_pct = ((forecast1['yhat'] - current_price) / current_price) * 100
    forecast2_pct = ((forecast2['yhat'] - current_price) / current_price) * 100

    fig.add_trace(go.Scatter(
        x=forecast1['ds'], y=forecast1['yhat'], mode='lines',
        name="Forecast 1 (Default)",
        line=dict(color='#ff7f0e', width=2), opacity=0.8,
        customdata=forecast1_pct,
        hovertemplate='<b>Forecast 1</b><br>Date: %{x}<br>Price: $%{y:.2f}<br>Change: %{customdata:+.1f}%<extra></extra>'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=forecast2['ds'], y=forecast2['yhat'], mode='lines',
        name="Forecast 2 (Higher Flexibility)",
        line=dict(color='#2ca02c', width=2), opacity=0.8,
        customdata=forecast2_pct,
        hovertemplate='<b>Forecast 2</b><br>Date: %{x}<br>Price: $%{y:.2f}<br>Change: %{customdata:+.1f}%<extra></extra>'
    ), row=1, col=1)

    # Confidence intervals
    fig.add_trace(go.Scatter(
        x=forecast1['ds'], y=forecast1['yhat_upper'], mode='lines',
        line=dict(width=0), showlegend=False, opacity=0.3
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=forecast1['ds'], y=forecast1['yhat_lower'], mode='lines',
        line=dict(width=0), name='Forecast 1 Confidence', fill='tonexty',
        fillcolor='rgba(255,127,14,0.1)', opacity=0.3
    ), row=1, col=1)

    # Clean chart without any horizontal lines or sliders
    slider_config = []

    # Forecast table
    table = go.Table(
        header=dict(
            values=['Period', 'Forecast 1 Price', 'Forecast 1 Change', 'Forecast 2 Price', 'Forecast 2 Change'],
            fill_color='lightblue', align='center'
        ),
        cells=dict(
            values=[
                ['Current', '30 Days', '90 Days', '1 Year'],
                [f"${current_price:.2f}", f"${forecast1.attrs['forecast_30d']:.2f}",
                 f"${forecast1.attrs['forecast_90d']:.2f}", f"${forecast1.attrs['forecast_365d']:.2f}"],
                ['--', f"{forecast1.attrs['pct_30d']:+.1f}%", f"{forecast1.attrs['pct_90d']:+.1f}%", f"{forecast1.attrs['pct_365d']:+.1f}%"],
                [f"${current_price:.2f}", f"${forecast2.attrs['forecast_30d']:.2f}",
                 f"${forecast2.attrs['forecast_90d']:.2f}", f"${forecast2.attrs['forecast_365d']:.2f}"],
                ['--', f"{forecast2.attrs['pct_30d']:+.1f}%", f"{forecast2.attrs['pct_90d']:+.1f}%", f"{forecast2.attrs['pct_365d']:+.1f}%"]
            ],
            fill_color='white', align='center'
        )
    )

    fig.add_trace(table, row=2, col=1)

    # Layout with vertical cursor line that follows mouse
    fig.update_layout(
        title=f"{symbol} Stock Analysis - Current Price: ${current_price:.2f}",
        template="plotly_white",
        height=950,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
        sliders=slider_config
    )

    # Add vertical and horizontal cursor lines with price labels
    fig.update_xaxes(
        showspikes=True,
        spikecolor="red",
        spikesnap="cursor",
        spikemode="across",
        spikethickness=2,
        spikedash="solid",
        row=1, col=1
    )

    fig.update_yaxes(
        showspikes=True,
        spikecolor="red",
        spikesnap="cursor",
        spikemode="across+toaxis",
        spikethickness=2,
        spikedash="solid",
        row=1, col=1
    )

    # Enhanced layout for better cursor appearance with price display
    fig.update_layout(
        spikedistance=1000,
        hoverdistance=100,
        hovermode="x unified"
    )

    # Add custom styling to show price values on y-axis when hovering
    fig.update_yaxes(
        tickformat="$,.2f",  # Format y-axis labels as currency
        showgrid=True,
        gridcolor="lightgray",
        gridwidth=1,
        row=1, col=1
    )

    return fig


def main(symbol="GOOG", period="5y", forecast_days=365):
    df, close_col = download_stock_data(symbol, period)
    if df is None:
        return None
    forecast1, forecast2 = create_prophet_forecasts(df, close_col, forecast_days)
    fig = create_interactive_chart(df, close_col, forecast1, forecast2, symbol)
    return fig


if __name__ == "__main__":
    SYMBOL = "GOOG"
    PERIOD = "5y"
    FORECAST_DAYS = 365
    fig = main(SYMBOL, PERIOD, FORECAST_DAYS)
    if fig:
        fig.show()

[*********************100%***********************]  1 of 1 completed
